In [8]:
%%configure -f
{
    "driverMemory": "2G",
    "executorMemory": "2G",
    "executorCores": 1,
    "numExecutors": 4
}

Starting Spark application


ID,YARN Application ID,Kind,State,Spark UI,Driver log,User,Current session?
1139,application_1765289937462_1132,pyspark,idle,Link,Link,None,✔


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

SparkSession available as 'spark'.


ID,YARN Application ID,Kind,State,Spark UI,Driver log,User,Current session?
1125,application_1765289937462_1118,pyspark,idle,Link,Link,None,
1128,application_1765289937462_1121,pyspark,idle,Link,Link,None,
1132,application_1765289937462_1125,pyspark,idle,Link,Link,None,
1134,application_1765289937462_1127,pyspark,idle,Link,Link,None,
1135,application_1765289937462_1128,pyspark,idle,Link,Link,None,
1136,application_1765289937462_1129,pyspark,idle,Link,Link,None,
1138,application_1765289937462_1131,pyspark,busy,Link,Link,None,
1139,application_1765289937462_1132,pyspark,idle,Link,Link,None,✔


In [9]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName("query2_python").getOrCreate()
sc = spark.sparkContext


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [10]:
crimes_old_df = spark.read.csv("s3://initial-notebook-data-bucket-dblab-905418150721/project_data/LA_Crime_Data/LA_Crime_Data_2010_2019.csv",\
    header=True,
    inferSchema=True)
crimes_new_df = spark.read.csv("s3://initial-notebook-data-bucket-dblab-905418150721/project_data/LA_Crime_Data/LA_Crime_Data_2020_2025.csv",\
    header=True,
    inferSchema=True)


descent_diction = spark.read.csv("s3://initial-notebook-data-bucket-dblab-905418150721/project_data/RE_codes.csv",\
    header=True,
    inferSchema=True)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [6]:
from pyspark.sql import functions as F
from pyspark.sql import Window
import time

runs = 10


# descent_diction.show(5,False)

year_df = crimes_df.withColumn(
    "year",
    F.year(F.to_timestamp("DATE OCC", "yyyy MMM dd hh:mm:ss a"))
)

year_df.select("year").show(2)

clean_df = year_df.filter(
    (F.col("year").isNotNull()) &
    (F.col("Vict Descent").isNotNull())
)

per_race = (
    clean_df
      .groupBy("year", "Vict Descent")
      .agg(F.count("*").alias("cnt"))
)

# per_race.show(5)

# Window: total victims per year

w_year = Window.partitionBy("year")

ranked = per_race.withColumn(
    "year_total",
    F.sum("cnt").over(w_year)
)

#  Percentage per year

ranked = ranked.withColumn(
    "percent",
    F.round(F.col("cnt") / F.col("year_total") * 100, 1)
)

#  Window: rank per year (top-3 per race)

w_rank = Window.partitionBy("year").orderBy(F.desc("cnt"))

ranked = ranked.withColumn(
    "rank",
    F.dense_rank().over(w_rank)
)

#  Keep only Top-3 per year

top3 = ranked.filter(F.col("rank") <= 3)


# top3.show(5,False)

#lets join with Racial codes

result = top3.join(
    descent_diction,
    on="Vict Descent",
    how="left"
)

# result.show(5,False)


#Final view

result = (
    result
      .select(
          "year",
          F.col("Vict Descent Full").alias("Vict_Descent"),
          "cnt",
          "percent"
      )
      .orderBy(F.desc("year"), F.desc("cnt"))
)

result.collect()


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+------------+-----------------+
|Vict Descent|Vict Descent Full|
+------------+-----------------+
|A           |Other Asian      |
|B           |Black            |
|C           |Chinese          |
|D           |Cambodian        |
|F           |Filipino         |
+------------+-----------------+
only showing top 5 rows

+----+
|year|
+----+
|2010|
|2010|
+----+
only showing top 2 rows

+----+------------+-----+
|year|Vict Descent|  cnt|
+----+------------+-----+
|2010|           V|   18|
|2010|           C|   82|
|2012|           F|  270|
|2012|           V|   13|
|2011|           B|32579|
+----+------------+-----+
only showing top 5 rows

+----+------------+-----+----------+-------+----+
|year|Vict Descent|cnt  |year_total|percent|rank|
+----+------------+-----+----------+-------+----+
|2010|H           |73558|188969    |38.9   |1   |
|2010|W           |53835|188969    |28.5   |2   |
|2010|B           |33937|188969    |18.0   |3   |
|2011|H           |70845|182588    |38.8   |1   |
|2

In [15]:
warmups = 3
runs = 10
times = []

for i in range(warmups + runs):

    crimes_df = crimes_old_df.unionByName(crimes_new_df)

    year_df = crimes_df.withColumn(
        "year",
        F.year(F.to_timestamp(F.col("DATE OCC"), "yyyy MMM dd hh:mm:ss a"))
    )

    clean_df = year_df.filter(
        F.col("year").isNotNull() & F.col("Vict Descent").isNotNull()
    )

    per_race = (
        clean_df.groupBy("year", "Vict Descent")
        .agg(F.count("*").alias("cnt"))
    )

    w_year = Window.partitionBy("year")
    ranked = per_race.withColumn("year_total", F.sum("cnt").over(w_year))

    ranked = ranked.withColumn(
        "percent",
        F.round(F.col("cnt") / F.col("year_total") * 100, 1)
    )

    w_rank = Window.partitionBy("year").orderBy(F.desc("cnt"))
    ranked = ranked.withColumn("rank", F.dense_rank().over(w_rank))

    top3 = ranked.filter(F.col("rank") <= 3)

    # πιο σταθερό join (αν το RE_codes είναι μικρό)
    result = top3.join(descent_diction, on="Vict Descent", how="left")

    result = (
        result.select(
            "year",
            F.col("Vict Descent Full").alias("Vict_Descent"),
            "cnt",
            "percent"
        )
        .orderBy(F.desc("year"), F.desc("cnt"))
    )

    start = time.time()
    rows = result.collect()
    dt = time.time() - start

    del rows
    gc.collect()

    if i < warmups:
        print(f"Warm-up {i+1}: {dt:.4f} sec")
    else:
        times.append(dt)
        print(f"Run {i-warmups+1}: {dt:.4f} sec")

mean_time = sum(times) / len(times)
print(f"Mean time (excluding warm-ups): {mean_time:.4f} sec")

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

Warm-up 1: 15.5399 sec
Warm-up 2: 8.8503 sec
Warm-up 3: 7.3012 sec
Run 1: 4.6137 sec
Run 2: 3.9090 sec
Run 3: 3.4271 sec
Run 4: 3.5693 sec
Run 5: 3.4043 sec
Run 6: 2.9797 sec
Run 7: 2.9439 sec
Run 8: 2.4479 sec
Run 9: 2.7335 sec
Run 10: 2.6416 sec
Mean time (excluding warm-ups): 3.2670 sec